In [ ]:
from google.cloud import bigquery
import pandas as pd

client = bigquery.Client()

query = """

WITH trans_table AS (
    SELECT
        AVG(transactions.gas_price/POWER(10,18)) AS avg_gas_price,
        AVG(transactions.value/POWER(10,18)) AS avg_transaction_value,
        blocks.timestamp AS tx_date,
        block_number
    FROM
        `bigquery-public-data.crypto_ethereum.transactions` AS transactions
    JOIN
        `bigquery-public-data.crypto_ethereum.blocks` AS blocks ON transactions.block_number = blocks.number
    WHERE
        blocks.timestamp >= '2023-12-01' AND blocks.timestamp <= '2024-03-01'
    GROUP BY
        tx_date, block_number
),
blocks_table AS (
    SELECT
        blocks.gas_used AS gas_used,
        blocks.transaction_count AS transaction_count,
        blocks.timestamp AS timestamp,
        blocks.number AS block_num
    FROM
        `bigquery-public-data.crypto_ethereum.blocks` AS blocks
    WHERE
        blocks.timestamp >= '2023-12-01' AND blocks.timestamp <= '2024-03-01'
),
tokens_table AS (
    SELECT
        value AS token_transfers_count,
        block_number,
        block_timestamp
    FROM
        `bigquery-public-data.crypto_ethereum.token_transfers` AS token_transfers
    WHERE
        block_timestamp >= '2023-12-01' AND block_timestamp <= '2024-03-01'
)
SELECT
    TIMESTAMP_ADD(TIMESTAMP_TRUNC(blocks_table.timestamp, HOUR), INTERVAL CAST(FLOOR(EXTRACT(MINUTE FROM blocks_table.timestamp) / 15) * 15 AS INT64) MINUTE) AS interval_start,
    AVG(CAST(trans_table.avg_transaction_value AS FLOAT64)) AS avg_transaction_value,
    AVG(CAST(trans_table.avg_gas_price AS FLOAT64)) AS avg_gas_price,
    AVG(CAST(tokens_table.token_transfers_count AS FLOAT64)) AS token_transfers_count,
    AVG(CAST(blocks_table.gas_used AS FLOAT64)) AS gas_used,
    AVG(CAST(blocks_table.transaction_count AS FLOAT64)) AS transaction_count
FROM
    trans_table
JOIN
    blocks_table ON trans_table.block_number = blocks_table.block_num
JOIN
    tokens_table ON tokens_table.block_number = blocks_table.block_num
GROUP BY
    interval_start
ORDER BY
    interval_start;


"""
query_job = client.query(query)

df = query_job.to_dataframe()



In [ ]:
# Convert DataFrame to CSV file
df.to_csv('output.csv', index=False)